In [1]:
import pandas as pd

In [2]:
df_from_moj = pd.read_csv("google_scraped_processed.csv")

In [3]:
df_from_wiki = pd.read_csv("wiki_prison_table.csv")

In [4]:
join_table = pd.read_csv("manually_tie_up_wiki_pop_figures.csv")
join_table = join_table.drop("remaining_from_wiki", axis=1)

In [5]:
merged = df_from_moj.merge(join_table, left_on="moj_prison_name", right_on="moj_prison_name", how="left")

In [6]:
df_from_doingtime = pd.read_csv("doingtime_data.csv")
df_from_doingtime.head()

,dt_name,dt_full_name,dt_region,dt_url,dt_address,dt_main_switchboard,dt_type_of_prison,dt_postcode,dt_lat,dt_lng
0,Altcourse,HMP Altcourse,NW,http://www.doingtime.co.uk/hmp-altcourse/,Higher Lane Fazakerley Liverpool L9 7LH,0151 522 2000,Male Cat B,L9 7LH,53.460927,-2.936031
1,Ashfield,HMP Ashfield,SW,http://www.doingtime.co.uk/hmp-ashfield/,"Shortwood Road, Pucklechurch, Bristol, BS16...",0117 303 8000,Male Cat C Sex Offenders,BS16 9QJ,51.482422,-2.438866
2,Askham Grange,HMP & YOI Askham Grange,Y&H,http://www.doingtime.co.uk/hmp-yoi-askham-grange/,Askham Richard York YO23 3FT,01904 772000,Female adult and young offender,YO23 3FT,53.927081,-1.185957
3,Aylesbury,HMYOI Aylesbury,SC,http://www.doingtime.co.uk/hmyoi-aylesbury/,Bierton Road Aylesbury Bucks HP20 1EH,01296 444 000,"Male Young Offenders 18-21yrs old, with senten...",HP20 1EH,51.821980,-0.799881
4,Bedford,HMP Bedford,EoE,http://www.doingtime.co.uk/hmp-bedford/,St Loyes Street Bedford MK40 1HG,01234 373000,Local Cat B Male,MK40 1HG,52.139321,-0.470232


In [7]:
merged = merged.merge(df_from_doingtime,left_on="dt_prison_name", right_on="dt_name", how="left")
final_merged = merged.merge(df_from_wiki, left_on="wiki_name", right_on="wiki_name", how="left")

In [8]:
final_merged.head()

,moj_prison_name,gscraped_desc,gscraped_address,gscraped_opened,gscraped_capacity,gscraped_type,gscraped_cat,gscraped_postcode,gscraped_lat,gscraped_lng,...,dt_type_of_prison,dt_postcode,dt_lat,dt_lng,wiki_location,wiki_county,wiki_operator,wiki_capacity,wiki_notes,wiki_category
0,Altcourse,HM Prison Altcourse is a Category B men's priv...,"Brookfield Dr, Liverpool L9 7LH",1997,"1,324","male, yoi",b,L9 7LH,53.460927,-2.936031,...,Male Cat B,L9 7LH,53.460927,-2.936031,Liverpool,Merseyside,G4S [2],1324 [4],Male Adults and Young Offenders [4],B
1,Ashfield,HMP Ashfield is now a Category C adult sex off...,Pucklechurch,1999,400,male,c,NaN,NaN,NaN,...,Male Cat C Sex Offenders,BS16 9QJ,51.482422,-2.438866,Pucklechurch,Gloucestershire,Serco [2],400 [5],Adult Sex Offenders,C
2,Askham Grange,HM Prison Askham Grange is a women's open cate...,"Askham Richard, York YO23 3FT",NaN,NaN,female,open,YO23 3FT,53.927081,-1.185957,...,Female adult and young offender,YO23 3FT,53.927081,-1.185957,Askham Richard,North Yorkshire,NaN,128 [6],Female Adults and Young Offenders [6],D
3,Aylesbury,HM Prison Aylesbury is a Young Offender Instit...,"Bierton Rd, Aylesbury HP20 1EH",1847,NaN,yoi,NaN,HP20 1EH,51.821980,-0.799881,...,"Male Young Offenders 18-21yrs old, with senten...",HP20 1EH,51.821980,-0.799881,Aylesbury,Buckinghamshire,NaN,443 [7],Young Offenders [7],HMYOI
4,Bedford,"HMP Bedford is a Category B men's prison, loca...","St Loyes St, Bedford MK40 1HG",1801,NaN,male,b,MK40 1HG,52.139321,-0.470232,...,Local Cat B Male,MK40 1HG,52.139321,-0.470232,Bedford,Bedfordshire,NaN,506 [8],NaN,B


In [36]:
pd.options.display.max_columns = 1000

def get_male_female(all_text):
    try:
        all_text = all_text.lower()
        response = []
        if " male" in all_text or " men" in all_text or re.search(r"^male", all_text):
            response.append("male")
            

        if "female" in all_text or "women" in all_text:
            response.append("female")

        return ", ".join(response)
    except:
        return ""
    
def get_youth(all_text):
    try:
        all_text = all_text.lower()
        response = []
        if "yo" in all_text or "youth" in all_text or "young" in all_text or "18" in all_text or "21" in all_text:
            response.append("youth")
        else:
            response.append("adult")
        return ", ".join(response)
    except:
        return ""

def get_category(all_text):
    cats = []
    try:
        all_text = all_text.lower()
        
        if "open prison" in all_text or "open category" in all_text:
            cats.append("open")
        
        if "closed" in all_text:
            cats.append("closed")
            
        a = re.search(r"cat (\w/\w |\w ?& ?\w |\w |\w)", all_text)
        
        if a:
            cats.append(a.group(1))
        
        return ", ".join(cats)
    except:
        return ""

final_merged["dt_male_female"] = final_merged["dt_type_of_prison"].apply(get_male_female)
final_merged["dt_youth"] = final_merged["dt_type_of_prison"].apply(get_youth)
final_merged["dt_category"] = final_merged["dt_type_of_prison"].apply(get_category)



In [37]:
final_merged


,moj_prison_name,gscraped_desc,gscraped_address,gscraped_opened,gscraped_capacity,gscraped_type,gscraped_cat,gscraped_postcode,gscraped_lat,gscraped_lng,dt_prison_name,wiki_name,remaining_from_dt,dt_name,dt_full_name,dt_region,dt_url,dt_address,dt_main_switchboard,dt_type_of_prison,dt_postcode,dt_lat,dt_lng,wiki_location,wiki_county,wiki_operator,wiki_capacity,wiki_notes,wiki_category,dt_male_female,dt_youth,dt_category
0,Altcourse,HM Prison Altcourse is a Category B men's priv...,"Brookfield Dr, Liverpool L9 7LH",1997,"1,324","male, yoi",b,L9 7LH,53.460927,-2.936031,Altcourse,Altcourse,Blantyre House,Altcourse,HMP Altcourse,NW,http://www.doingtime.co.uk/hmp-altcourse/,Higher Lane Fazakerley Liverpool L9 7LH,0151 522 2000,Male Cat B,L9 7LH,53.460927,-2.936031,Liverpool,Merseyside,G4S [2],1324 [4],Male Adults and Young Offenders [4],B,male,adult,b
1,Ashfield,HMP Ashfield is now a Category C adult sex off...,Pucklechurch,1999,400,male,c,NaN,NaN,NaN,Ashfield,Ashfield,Peterborough Female,Ashfield,HMP Ashfield,SW,http://www.doingtime.co.uk/hmp-ashfield/,"Shortwood Road, Pucklechurch, Bristol, BS16...",0117 303 8000,Male Cat C Sex Offenders,BS16 9QJ,51.482422,-2.438866,Pucklechurch,Gloucestershire,Serco [2],400 [5],Adult Sex Offenders,C,male,adult,c
2,Askham Grange,HM Prison Askham Grange is a women's open cate...,"Askham Richard, York YO23 3FT",NaN,NaN,female,open,YO23 3FT,53.927081,-1.185957,Askham Grange,Askham Grange,Prescoed,Askham Grange,HMP & YOI Askham Grange,Y&H,http://www.doingtime.co.uk/hmp-yoi-askham-grange/,Askham Richard York YO23 3FT,01904 772000,Female adult and young offender,YO23 3FT,53.927081,-1.185957,Askham Richard,North Yorkshire,NaN,128 [6],Female Adults and Young Offenders [6],D,female,youth,
3,Aylesbury,HM Prison Aylesbury is a Young Offender Instit...,"Bierton Rd, Aylesbury HP20 1EH",1847,NaN,yoi,NaN,HP20 1EH,51.821980,-0.799881,Aylesbury,Aylesbury,Spring Hill,Aylesbury,HMYOI Aylesbury,SC,http://www.doingtime.co.uk/hmyoi-aylesbury/,Bierton Road Aylesbury Bucks HP20 1EH,01296 444 000,"Male Young Offenders 18-21yrs old, with senten...",HP20 1EH,51.821980,-0.799881,Aylesbury,Buckinghamshire,NaN,443 [7],Young Offenders [7],HMYOI,male,youth,
4,Bedford,"HMP Bedford is a Category B men's prison, loca...","St Loyes St, Bedford MK40 1HG",1801,NaN,male,b,MK40 1HG,52.139321,-0.470232,Bedford,Bedford,NaN,Bedford,HMP Bedford,EoE,http://www.doingtime.co.uk/hmp-bedford/,St Loyes Street Bedford MK40 1HG,01234 373000,Local Cat B Male,MK40 1HG,52.139321,-0.470232,Bedford,Bedfordshire,NaN,506 [8],NaN,B,male,adult,b
5,Belmarsh,HM Prison Belmarsh is a Category A men's priso...,"Western Way, London SE28 0EB",1991,NaN,male,a,SE28 0EB,51.496846,0.092704,Belmarsh,Belmarsh,NaN,Belmarsh,HMP Belmarsh,Lon,http://www.doingtime.co.uk/hmp-belmarsh/,Western Way Thamesmead London SE28 0EB,020 8331 4400,Local male prison with high secure unit.,SE28 0EB,51.496846,0.092704,Thamesmead,London,NaN,910 [9],NaN,A,male,adult,
6,Birmingham,HM Prison Winson Green is a Category B/C men's...,"Winson Green Rd, Birmingham B18 4AS",1849,"1,450",male,b/c,B18 4AS,52.492489,-1.938513,Birmingham,Birmingham,NaN,Birmingham,HMP Birmingham,WM,http://www.doingtime.co.uk/hmp-birmingham/,"Winson Green Road, Birmingham, B18 4AS",0121 598 8000,Male Cat B & C,B18 4AS,52.492489,-1.938513,Birmingham,West Midlands,G4S [2],1450 [10],Previously known as Winson Green. [11],"B, C",male,adult,b
7,Brinsford,HM Prison Brinsford is a male juveniles' priso...,"New Rd, Wolverhampton WV10 7PY",1991,NaN,"male, yoi",NaN,WV10 7PY,52.650142,-2.110816,Brinsford,Brinsford,NaN,Brinsford,HMYOI Brinsford,WM,http://www.doingtime.co.uk/hmp-brinsford/,"New Road, Featherstone, Wolverhampton, WV10 7PY",01902 533 450,Male Young Offenders (18-21),WV10 7PY,52.650142,-2.110816,Featherstone,Staffordshire,NaN,569 [14],Young Offenders [14],HMYOI,male,youth,
8,Bristol,"HMP Bristol is a Category B men's prison, loca...",NaN,NaN,NaN,male,b,NaN,NaN,NaN,Bristol,Bristol,NaN,Bristol,HMP Bristol,SW,http://www.do

In [38]:
final_merged.columns = [c.lower() for c in final_merged.columns]

In [41]:
final_merged.to_csv("all_prison_data.csv", index=False, encoding="utf-8")